In [1]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 898.5/898.5 kB 19.4 MB/s eta 0:00:00


In [2]:
!pip install opencv-python

In [3]:
!pip install deep-sort-realtime

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 65.2 MB/s eta 0:00:00


In [4]:
!pip install ultralytics opencv-python deep-sort-realtime


In [5]:
import cv2
from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort
import numpy as np

# Load YOLO model
model = YOLO('yolov8n.pt')  # Load YOLOv8 pre-trained model

# Initialize DeepSORT tracker
tracker = DeepSort(max_age=30, n_init=2, nn_budget=100)

# Define the whole boundary and lanes
main_boundary = [(750, 983), (1708, 705), (1532, 605), (676, 785)]
lanes = {
    'Lane 1': [[1273, 832], [1707, 705], [1533, 606], [1141, 689]],
    'Lane 2': [[751, 983], [1273, 831], [1141, 688], [676, 786]],

}

# Initialize lane vehicle count and tracking state
lane_counts = {lane: 0 for lane in lanes.keys()}
vehicle_states = {}  # To store each track_id and lane states
total_vehicles = 0  # Total vehicle count across all lanes

def is_point_in_polygon(point, polygon):
    """Check if a point is inside a polygon using cv2.pointPolygonTest."""
    polygon_np = np.array(polygon, np.int32)
    return cv2.pointPolygonTest(polygon_np, point, False) >= 0

# Load the video file
video_path = '/content/drive/MyDrive/detection_tracking_counting/video_1.mp4'
cap = cv2.VideoCapture(video_path)

# Get video properties
fps = cap.get(cv2.CAP_PROP_FPS)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Define the codec and create VideoWriter object
output_path = 'output_video_1.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Perform detection using YOLO
    results = model(frame)
    detections = []

    for result in results:
        for box in result.boxes.data:  # Extract bounding box info
            x1, y1, x2, y2, conf, cls = map(float, box[:6])
            cls = int(cls)  # Class ID
            if cls == 2:  # Class ID for car (2) only
                detections.append([[x1, y1, x2, y2], conf])  # Format for DeepSORT

    # Update tracker with detections
    tracks = tracker.update_tracks(detections, frame=frame)

    for track in tracks:
        if not track.is_confirmed():
            continue

        x1, y1, x2, y2 = map(int, track.to_ltwh())
        track_id = track.track_id
        cx, cy = (x1 + x2) // 2, (y1 + y2) // 2  # Center of bounding box

        # Verify vehicle starts inside the main boundary and detect lane entries
        if is_point_in_polygon((cx, cy), main_boundary):
            for lane_name, polygon in lanes.items():
                inside_lane = is_point_in_polygon((cx, cy), polygon)
                prev_state = vehicle_states.get(track_id, {}).get(lane_name, False)

                if inside_lane and not prev_state:
                    lane_counts[lane_name] += 1
                    total_vehicles += 1  # Increment total count
                    vehicle_states.setdefault(track_id, {})[lane_name] = True
                elif not inside_lane:
                    vehicle_states.setdefault(track_id, {})[lane_name] = False  # Reset when outside

        # Draw bounding boxes and track ID
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(frame, f'ID {track_id}', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

    # Draw polygons and counts for all lanes and boundary
    cv2.polylines(frame, [np.array(main_boundary, np.int32)], True, (255, 255, 0), 2)
    for lane_name, polygon in lanes.items():
        cv2.polylines(frame, [np.array(polygon, np.int32)], True, (0, 0, 255), 2)
        cv2.putText(frame, f'{lane_name}: {lane_counts[lane_name]}',
                    (polygon[0][0], polygon[0][1] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)

    # Display total vehicle count on the frame
    cv2.putText(frame, f'Total Vehicles: {total_vehicles}', (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2)

    # Write the frame to output
    out.write(frame)

cap.release()
out.release()

# Print total counts for each lane
print("Final Lane Counts:")
for lane, count in lane_counts.items():
    print(f'{lane}: {count}')

# Print total vehicle count across all lanes
print(f'Total Vehicles: {total_vehicles}')


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


100%|██████████| 6.25M/6.25M [00:00<00:00, 106MB/s]



0: 384x640 7 cars, 1 bus, 3 trucks, 91.7ms
Speed: 6.9ms preprocess, 91.7ms inference, 790.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 cars, 2 buss, 1 truck, 10.5ms
Speed: 4.7ms preprocess, 10.5ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 8 cars, 2 buss, 1 truck, 14.9ms
Speed: 3.0ms preprocess, 14.9ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 cars, 1 bus, 1 truck, 8.9ms
Speed: 3.2ms preprocess, 8.9ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 cars, 2 trucks, 19.1ms
Speed: 3.0ms preprocess, 19.1ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 cars, 1 bus, 1 truck, 10.2ms
Speed: 3.1ms preprocess, 10.2ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 8 cars, 1 bus, 1 truck, 11.6ms
Speed: 6.2ms preprocess, 11.6ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 9 cars, 2 t